# This notebook is deticated to create recommendations engines

## Importing libraries

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

## Loading Tables

- Loading the selected tables from data store to apply for recommendations engines

In [22]:
event = pd.read_csv('medalion_data_store/bronze/events.csv')
offer_rank = pd.read_csv('medalion_data_store/silver/offer_event_time.csv')

user_item_matrix = pd.read_csv('medalion_data_store/silver/person_event_r_time.csv')

In [23]:
offer_rank

,ofr_id_short,offer_type,difficulty,cnt_completed,cnt_received,cnt_viewed,time_mean_completed,time_mean_received,time_mean_viewed,comp_ratio,view_ratio,comp-vie-intensity
0,ofr_A,bogo,10,3688.0,7658.0,6716.0,394.767,329.760,352.622,0.482,0.877,0.549
1,ofr_B,bogo,10,3331.0,7593.0,7298.0,385.722,335.153,353.119,0.439,0.961,0.456
2,ofr_C,informational,0,0.0,7617.0,4144.0,NaN,331.885,358.639,NaN,0.544,NaN
3,ofr_D,bogo,5,4354.0,7677.0,4171.0,407.051,334.146,361.977,0.567,0.543,1.044
4,ofr_E,discount,20,3420.0,7668.0,2663.0,431.549,331.336,366.748,0.446,0.347,1.284
5,ofr_F,discount,7,5156.0,7646.0,7337.0,400.318,336.377,354.750,0.674,0.960,0.703
6,ofr_G,discount,10,5317.0,7597.0,7327.0,399.117,330.487,348.868,0.700,0.964,0.726
7,ofr_H,informational,0,0.0,7618.0,6687.0,NaN,332.475,353.934,NaN,0.878,NaN
8,ofr_I,bogo,5,4296.0,7571.0,7264.0,382.936,332.171,349.797,0.567,0.959,0.591
9,ofr_J,discount,10,4017.0,7632.0,4118.0,409.952,332.003,356.204,0.526,0.540,0.975


# Rank-Based Recommendations

Rank-Based Recommendation is a -simple- yet effective approach that ranks items based on predefined criteria and suggests the top-ranked items to users.

## Steps

---

### 1. Ranking Offers Using `offer_rank` dataset choosed and loaded.
- Define a ranking criterion, such as:
  - User engagement metrics (e.g., offer completed, completion rate).
- Sort the offers based on the selected ranking metric.

### 2. Generating Recommendations
- Use the ranked offers table along with the event data to personalize recommendations:
  - Filter relevant offers.
  - Select the top-ranked items to generate a recommendation list.
  - Ensure that only previously unseen are included in the recommendation.

---

### Defining a Rank-Based recommendation function

In [24]:
# Function to recommend offers based on ranking and past interactions
def offer_rank_recommendation(person_id, event, offer_rank):
    """
    Recommends offers that the person has not seen yet,
    prioritizing those with higher influence and completion rates.
    
    Parameters:
    person_id (str): The unique identifier of the person.
    event (pd.DataFrame): The event data containing past interactions.
    offer_rank (pd.DataFrame): The ranked offers data.
    
    Returns:
    list: A list of recommended offer IDs sorted by priority.
    """
    # Get offers that the person has already seen
    products_seen = event[event['person'] == person_id]['ofr_id_short'].unique()

    # Filter out seen offers and sort remaining offers by influence and completion rate
    products_not_seen = (
        offer_rank[~offer_rank['ofr_id_short'].isin(products_seen)]
        .sort_values(by=['comp_ratio', 'view_ratio'], ascending=False)['ofr_id_short']
        .tolist()
    )

    print('*** Rank-Based Recommendations ***\n')
    print(f'The person saw: {products_seen}')
    print(f'The recommended offers to person {person_id} are: {products_not_seen}')
    print('------------------\n')
    
    return products_not_seen  # Optionally, return only the top N recommendations

### Checking for a user

In [25]:
# Example usage
r1 = pd.Series(offer_rank_recommendation('01d26f638c274aa0b965d24cefe3183f', event, offer_rank), name='01d26f638c274aa0b965d24cefe3183f')

*** Rank-Based Recommendations ***

The person saw: ['ofr_E' 'ofr_C' 'ofr_H']
The recommended offers to person 01d26f638c274aa0b965d24cefe3183f are: ['ofr_G', 'ofr_F', 'ofr_I', 'ofr_D', 'ofr_J', 'ofr_A', 'ofr_B']
------------------



In [26]:
r2 = pd.Series(offer_rank_recommendation('ffff82501cea40309d5fdd7edcca4a07', event, offer_rank), name='ffff82501cea40309d5fdd7edcca4a07')

*** Rank-Based Recommendations ***

The person saw: ['ofr_G' 'ofr_E' 'ofr_J' 'ofr_D']
The recommended offers to person ffff82501cea40309d5fdd7edcca4a07 are: ['ofr_F', 'ofr_I', 'ofr_A', 'ofr_B', 'ofr_H', 'ofr_C']
------------------



# User-User Collaborative Filtering

User-User Collaborative Filtering is a recommendation technique that finds similar users based on their interactions and preferences or segment to suggest items.

## Steps

--- 

### 1. Creating a User-Item Matrix
- Creating the matrix where rows represent users, and columns represent items.
- Each cell contains a value representing the interaction between the user and the item AND his segment (gender, age group).

> note: Segmented user similarity can be done choosing features targuet in the dataset `user_item_profile`.

### 2. Scaling the Data
- Since different users may have different interaction levels, scaling is necessary.
- Apply an appropriate scaler depending on the data type:
  - **StandardScaler**: If data follows a normal distribution.
  - **MinMaxScaler**: If values need to be normalized between a range.

### 3. Calculating Cosine Similarity
- Compute the pairwise similarity between users using **Cosine Similarity**.

- This generates a **User-User Similarity Matrix**, where each entry \((i, j)\) represents the similarity between users \(i\) and \(j\).

### 4. Generating Recommendations
- For a target user:
  - Identify the most similar users (neighbors) based on the similarity matrix.
  - Aggregate their interactions to predict scores for items the target user has not interacted with.
  - Rank items based on predicted scores and recommend the highest-ranked items.

---

### Creating USER-ITEM matrix

In [27]:
user_item_matrix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14820 entries, 0 to 14819
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   person               14820 non-null  object 
 1   gender               14820 non-null  object 
 2   age_group            14820 non-null  object 
 3   cnt_completed_ofr_G  14820 non-null  float64
 4   cnt_completed_ofr_I  14820 non-null  float64
 5   cnt_completed_ofr_J  14820 non-null  float64
 6   cnt_received_ofr_C   14820 non-null  float64
 7   cnt_received_ofr_G   14820 non-null  float64
 8   cnt_received_ofr_H   14820 non-null  float64
 9   cnt_received_ofr_I   14820 non-null  float64
 10  cnt_received_ofr_J   14820 non-null  float64
 11  cnt_viewed_ofr_C     14820 non-null  float64
 12  cnt_viewed_ofr_G     14820 non-null  float64
 13  cnt_viewed_ofr_H     14820 non-null  float64
 14  cnt_viewed_ofr_I     14820 non-null  float64
 15  cnt_completed_ofr_D  14820 non-null 

In [28]:
pd.get_dummies(user_item_matrix.iloc[:,1:].dropna(), dtype=int)

,cnt_completed_ofr_G,cnt_completed_ofr_I,cnt_completed_ofr_J,cnt_received_ofr_C,cnt_received_ofr_G,cnt_received_ofr_H,cnt_received_ofr_I,cnt_received_ofr_J,cnt_viewed_ofr_C,cnt_viewed_ofr_G,...,cnt_completed_ofr_A,cnt_viewed_ofr_A,cnt_viewed_ofr_J,gender_F,gender_M,gender_O,age_group_Adult,age_group_Middle,age_group_Senior,age_group_Young
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0,1,0,1,0,0,0
1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0,0,1,1,0,0,0
2,2.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1,0,0,0,1,0,0
3,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,0,0,0,0,0,1
4,2.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,1.0,2.0,...,0.0,0.0,0.0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14815,1.0,0.0,1.0,0.0,1.0,2.0,0.0,2.0,0.0,1.0,...,0.0,0.0,0.0,1,0,0,0,0,1,0
14816,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,...,0.0,1.0,0.0,0,1,0,0,0,1,0
14817,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0,1,0,0,0,1,0
14818,0.0,2.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,1,0,1,0,0,0


In [29]:
# creating user-item matrix choosing all feature in user_item_matrix
user_item_matrix_dummies = pd.get_dummies(user_item_matrix.iloc[:,1:].dropna(), dtype=int)

# setting index as person id
user_item_matrix_dummies.index = user_item_matrix.dropna()['person']

user_item_matrix_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14820 entries, 0009655768c64bdeb2e877511632db8f to ffff82501cea40309d5fdd7edcca4a07
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cnt_completed_ofr_G  14820 non-null  float64
 1   cnt_completed_ofr_I  14820 non-null  float64
 2   cnt_completed_ofr_J  14820 non-null  float64
 3   cnt_received_ofr_C   14820 non-null  float64
 4   cnt_received_ofr_G   14820 non-null  float64
 5   cnt_received_ofr_H   14820 non-null  float64
 6   cnt_received_ofr_I   14820 non-null  float64
 7   cnt_received_ofr_J   14820 non-null  float64
 8   cnt_viewed_ofr_C     14820 non-null  float64
 9   cnt_viewed_ofr_G     14820 non-null  float64
 10  cnt_viewed_ofr_H     14820 non-null  float64
 11  cnt_viewed_ofr_I     14820 non-null  float64
 12  cnt_completed_ofr_D  14820 non-null  float64
 13  cnt_completed_ofr_E  14820 non-null  float64
 14  cnt_completed_ofr_F  14820 non-nu

### Scalling dataset

- As scaler is sensible to data distribution, it will show if there is a normal aspect

- choosing MinMaxScaler as scaler function

In [30]:
df = user_item_matrix_dummies

scaler = MinMaxScaler()

df = scaler.fit_transform(df)

df = pd.DataFrame(df, index=user_item_matrix_dummies.index, columns=user_item_matrix_dummies.columns)

df = df.astype({col: 'float16' for col in df.select_dtypes('float64').columns})

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14820 entries, 0009655768c64bdeb2e877511632db8f to ffff82501cea40309d5fdd7edcca4a07
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cnt_completed_ofr_G  14820 non-null  float16
 1   cnt_completed_ofr_I  14820 non-null  float16
 2   cnt_completed_ofr_J  14820 non-null  float16
 3   cnt_received_ofr_C   14820 non-null  float16
 4   cnt_received_ofr_G   14820 non-null  float16
 5   cnt_received_ofr_H   14820 non-null  float16
 6   cnt_received_ofr_I   14820 non-null  float16
 7   cnt_received_ofr_J   14820 non-null  float16
 8   cnt_viewed_ofr_C     14820 non-null  float16
 9   cnt_viewed_ofr_G     14820 non-null  float16
 10  cnt_viewed_ofr_H     14820 non-null  float16
 11  cnt_viewed_ofr_I     14820 non-null  float16
 12  cnt_completed_ofr_D  14820 non-null  float16
 13  cnt_completed_ofr_E  14820 non-null  float16
 14  cnt_completed_ofr_F  14820 non-nu

### Creating user similarity matrix

In [31]:
# Compute user similarity
user_similarity_df = None
user_similarity = cosine_similarity(df)
user_similarity_df = pd.DataFrame(user_similarity, index=df.index, columns=df.index)
user_similarity_df

person,0009655768c64bdeb2e877511632db8f,0011e0d4e6b944f998e987f904e8c1e5,0020c2b971eb4e9188eac86d93036a77,0020ccbbb6d84e358d3414a3ff76cffd,003d66b6608740288d6cc97a6903f4f0,00426fe3ffde4c6b9cb9ad6d077a13ea,004b041fbfe44859945daa2c7f79ee64,004c5799adbf42868b9cff0396190900,005500a7188546ff8a767329a2f7c76a,0056df74b63b4298809f0b375a304cf4,...,ffe5257abf8840b395e1ee6b29894637,ffeaa02452ef451082a0361c3ca62ef5,ffed75d3abc64b488982f50ed12878b5,fff0f0aac6c547b9b263080f09a5586a,fff29fb549084123bd046dbc5ceb4faa,fff3ba4757bd42088c044ca26d73817a,fff7576017104bcc8677a8d63322b5e1,fff8957ea8b240a6b5e634b6ee8eafcf,fffad4f4828548d1b5583907f2e9906b,ffff82501cea40309d5fdd7edcca4a07
person,,,,,,,,,,,,,,,,,,,,,
0009655768c64bdeb2e877511632db8f,1.000000,0.434304,0.140217,0.101223,0.544732,0.149118,0.171517,0.559658,0.354722,0.419838,...,0.460076,0.138009,0.405085,0.468795,0.119716,0.176985,0.472623,0.474493,0.859199,0.456546
0011e0d4e6b944f998e987f904e8c1e5,0.434304,1.000000,0.027889,0.165907,0.456396,0.044489,0.023729,0.000000,0.040740,0.237182,...,0.150980,0.052561,0.000000,0.091529,0.000000,0.107323,0.041944,0.024379,0.431805,0.413483
0020c2b971eb4e9188eac86d93036a77,0.140217,0.027889,1.000000,0.390374,0.568827,0.511972,0.870287,0.520133,0.383638,0.361126,...,0.014297,0.398250,0.073498,0.045364,0.860767,0.495666,0.270249,0.120830,0.027003,0.371102
0020ccbbb6d84e358d3414a3ff76cffd,0.101223,0.165907,0.390374,1.000000,0.376716,0.799575,0.468640,0.123116,0.042222,0.141589,...,0.114951,0.395097,0.409841,0.000000,0.381111,0.473016,0.043470,0.000000,0.229326,0.351049
003d66b6608740288d6cc97a6903f4f0,0.544732,0.456396,0.568827,0.376716,1.000000,0.530421,0.522062,0.223082,0.000000,0.096197,...,0.079717,0.364667,0.000000,0.087554,0.406096,0.499195,0.240733,0.116603,0.351946,0.689234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff3ba4757bd42088c044ca26d73817a,0.176985,0.107323,0.495666,0.473016,0.499195,0.573133,0.447295,0.115686,0.080837,0.067770,...,0.520764,0.852642,0.000000,0.363230,0.363753,1.000000,0.499359,0.435369,0.103915,0.548296
fff7576017104bcc8677a8d63322b5e1,0.472623,0.041944,0.270249,0.043470,0.240733,0.132649,0.135775,0.537474,0.466217,0.390859,...,0.731101,0.352611,0.471405,0.741941,0.239235,0.499359,1.000000,0.883482,0.365508,0.136010
fff8957ea8b240a6b5e634b6ee8eafcf,0.474493,0.024379,0.120830,0.000000,0.116603,0.051400,0.078918,0.430897,0.361311,0.429121,...,0.799894,0.364357,0.493197,0.913223,0.128356,0.435369,0.883482,1.000000,0.377684,0.039527


### Definning a User-User colaborative filtering recommendation function

In [40]:
def recommend_user_based(user_id, user_similarity_df, event):
    """
    Recommend products to a user based on similar users' preferences.
    
    Parameters:
        user_id (int/str): The ID of the target user.
        user_similarity_df (pd.DataFrame): DataFrame containing user similarity scores.
        event (pd.DataFrame): DataFrame containing user interactions with products.
    
    Returns:
        list: List of recommended product IDs.
    """
    print('*** User-User Collaborative Filtering ***\n')

    if user_id not in user_similarity_df.index:
        return ['the user is not in the similarity matrix']  # Return an empty list if the user is not in the similarity matrix



    recommendations = []
    i = 0
    while recommendations == [] or len(recommendations) < 3:
        
        # Get products that user have seem
        user_products = {x for x in event[event['person'] == user_id]['ofr_id_short']}

        # Get similar users, excluding the current user
        similar_users = user_similarity_df[user_id].drop(index=user_id).sort_values(ascending=False)
        most_similar_user = similar_users.index[i]  # Retrieve the most similar user

        # Retrieve the products interacted by most similar users
        similar_user_products = {x for x in event[event['person'] == most_similar_user]['ofr_id_short']}

        # ranking the products of the most similar user
        ranked_similar_user_products = (offer_rank.loc[offer_rank['ofr_id_short'].isin(similar_user_products),:]
                                        .sort_values(by=['comp_ratio', 'view_ratio'], ascending=False)['ofr_id_short']
                                        ).tolist()
        
        # Recommend products that the similar user has interacted with but the current user hasn't
        recommendations = [x for x in ranked_similar_user_products if x not in user_products]
        i += 1
    
    print(f'Current user ({user_id}) products: {user_products}\n')
    print(f'First one Similar user found: {most_similar_user}')
    print(f'Products of first one similar user ({most_similar_user}): {similar_user_products}')
    print(f'Three best Recommendation(s) for current user: {recommendations}')
    print('------------------\n')

    return recommendations



## Cheking for two persons

In [41]:
u1 = pd.Series(recommend_user_based('01d26f638c274aa0b965d24cefe3183f', user_similarity_df, event), name='01d26f638c274aa0b965d24cefe3183f')

*** User-User Collaborative Filtering ***

Current user (01d26f638c274aa0b965d24cefe3183f) products: {'ofr_C', 'ofr_E', 'ofr_H'}

First one Similar user found: 2ca6147969524c5a8ce2b48aade075ab
Products of first one similar user (2ca6147969524c5a8ce2b48aade075ab): {'ofr_C', 'ofr_E', 'ofr_J', 'ofr_B', 'ofr_F', 'ofr_H'}
Three best Recommendation(s) for current user: ['ofr_F', 'ofr_J', 'ofr_B']
------------------



In [42]:
u2 = pd.Series(recommend_user_based('2cb4f97358b841b9a9773a7aa05a9d77', user_similarity_df, event), name='2cb4f97358b841b9a9773a7aa05a9d77')

*** User-User Collaborative Filtering ***

Current user (2cb4f97358b841b9a9773a7aa05a9d77) products: {'ofr_H', 'ofr_D', 'ofr_C'}

First one Similar user found: 285fc1b40f194fafbe95c9cfbc6b3d6f
Products of first one similar user (285fc1b40f194fafbe95c9cfbc6b3d6f): {'ofr_E', 'ofr_A', 'ofr_I', 'ofr_D', 'ofr_H'}
Three best Recommendation(s) for current user: ['ofr_I', 'ofr_A', 'ofr_E']
------------------



In [43]:
# Create a list of users
person_list = list({x for x in user_item_matrix['person']})
person_list = person_list[:10]
person_list

['42de30f65a554db2b2928ab318de7390',
 'd3c01d3f66d142df959c19fb8bef8af9',
 '02e2211319524b9e8bd09794d00d8513',
 '1a43f12c497f4ea0aa3897252b84d3dd',
 '25c89cd833a04d51b76510b1ae335476',
 '3504fbcd34474688ac705a18dbf6f9f9',
 '9d9762e2f526402aa214e18be86ecb0f',
 'cd0cb9d1599c47a0bb69c49d19c2117f',
 'e2a8952a0a504021995b8c609ebd6ded',
 '2e792741fcd548889f45c3399a1d6f69']

In [44]:
# Create recommendations tables
rank_recom = []
user_recom = []
for person in person_list:
    r = offer_rank_recommendation(person, event, offer_rank)
    u = recommend_user_based(person, user_similarity_df, event)
    rank_recom.append(r)
    user_recom.append(u)

rank_recom = pd.DataFrame(rank_recom).fillna('-')
rank_recom.columns = [f'ofr_rec_{i}' for i in range(rank_recom.shape[1])]
rank_recom.index = person_list


user_recom = pd.DataFrame(user_recom).fillna('-')
user_recom.columns = [f'ofr_rec_{i}' for i in range(user_recom.shape[1])]
user_recom.index = person_list


*** Rank-Based Recommendations ***

The person saw: ['ofr_A' 'ofr_J' 'ofr_E']
The recommended offers to person 42de30f65a554db2b2928ab318de7390 are: ['ofr_G', 'ofr_F', 'ofr_I', 'ofr_D', 'ofr_B', 'ofr_H', 'ofr_C']
------------------

*** User-User Collaborative Filtering ***

Current user (42de30f65a554db2b2928ab318de7390) products: {'ofr_A', 'ofr_E', 'ofr_J'}

First one Similar user found: 1c9477cf75a24b52af9918938d19d93c
Products of first one similar user (1c9477cf75a24b52af9918938d19d93c): {'ofr_I', 'ofr_A', 'ofr_J', 'ofr_B', 'ofr_D'}
Three best Recommendation(s) for current user: ['ofr_I', 'ofr_D', 'ofr_B']
------------------

*** Rank-Based Recommendations ***

The person saw: ['ofr_E' 'ofr_F' 'ofr_C' 'ofr_J']
The recommended offers to person d3c01d3f66d142df959c19fb8bef8af9 are: ['ofr_G', 'ofr_I', 'ofr_D', 'ofr_A', 'ofr_B', 'ofr_H']
------------------

*** User-User Collaborative Filtering ***

Current user (d3c01d3f66d142df959c19fb8bef8af9) products: {'ofr_J', 'ofr_C', 'ofr_E', '

In [37]:
rank_recom

,ofr_rec_0,ofr_rec_1,ofr_rec_2,ofr_rec_3,ofr_rec_4,ofr_rec_5,ofr_rec_6
42de30f65a554db2b2928ab318de7390,ofr_G,ofr_F,ofr_I,ofr_D,ofr_B,ofr_H,ofr_C
d3c01d3f66d142df959c19fb8bef8af9,ofr_G,ofr_I,ofr_D,ofr_A,ofr_B,ofr_H,-
02e2211319524b9e8bd09794d00d8513,ofr_G,ofr_I,ofr_J,ofr_E,ofr_B,ofr_C,-
1a43f12c497f4ea0aa3897252b84d3dd,ofr_G,ofr_F,ofr_D,ofr_J,ofr_A,ofr_H,ofr_C
25c89cd833a04d51b76510b1ae335476,ofr_F,ofr_I,ofr_D,ofr_E,ofr_B,ofr_C,-
3504fbcd34474688ac705a18dbf6f9f9,ofr_F,ofr_I,ofr_J,ofr_E,ofr_B,ofr_H,-
9d9762e2f526402aa214e18be86ecb0f,ofr_G,ofr_I,ofr_D,ofr_J,ofr_E,ofr_H,-
cd0cb9d1599c47a0bb69c49d19c2117f,ofr_G,ofr_F,ofr_A,ofr_E,ofr_H,ofr_C,-
e2a8952a0a504021995b8c609ebd6ded,ofr_G,ofr_I,ofr_J,ofr_A,ofr_E,ofr_H,-
2e792741fcd548889f45c3399a1d6f69,ofr_A,ofr_E,ofr_B,ofr_H,ofr_C,-,-


In [45]:
user_recom

,ofr_rec_0,ofr_rec_1,ofr_rec_2
42de30f65a554db2b2928ab318de7390,ofr_I,ofr_D,ofr_B
d3c01d3f66d142df959c19fb8bef8af9,ofr_G,ofr_B,ofr_H
02e2211319524b9e8bd09794d00d8513,ofr_J,ofr_E,ofr_C
1a43f12c497f4ea0aa3897252b84d3dd,ofr_D,ofr_J,ofr_C
25c89cd833a04d51b76510b1ae335476,ofr_F,ofr_I,ofr_E
3504fbcd34474688ac705a18dbf6f9f9,ofr_E,ofr_B,ofr_H
9d9762e2f526402aa214e18be86ecb0f,ofr_D,ofr_E,ofr_H
cd0cb9d1599c47a0bb69c49d19c2117f,ofr_G,ofr_F,ofr_E
e2a8952a0a504021995b8c609ebd6ded,ofr_G,ofr_A,ofr_H
2e792741fcd548889f45c3399a1d6f69,ofr_B,ofr_H,ofr_C


In [39]:
print(rank_recom.to_markdown())  # or df.to_latex()

|                                  | ofr_rec_0   | ofr_rec_1   | ofr_rec_2   | ofr_rec_3   | ofr_rec_4   | ofr_rec_5   | ofr_rec_6   |
|:---------------------------------|:------------|:------------|:------------|:------------|:------------|:------------|:------------|
| 42de30f65a554db2b2928ab318de7390 | ofr_G       | ofr_F       | ofr_I       | ofr_D       | ofr_B       | ofr_H       | ofr_C       |
| d3c01d3f66d142df959c19fb8bef8af9 | ofr_G       | ofr_I       | ofr_D       | ofr_A       | ofr_B       | ofr_H       | -           |
| 02e2211319524b9e8bd09794d00d8513 | ofr_G       | ofr_I       | ofr_J       | ofr_E       | ofr_B       | ofr_C       | -           |
| 1a43f12c497f4ea0aa3897252b84d3dd | ofr_G       | ofr_F       | ofr_D       | ofr_J       | ofr_A       | ofr_H       | ofr_C       |
| 25c89cd833a04d51b76510b1ae335476 | ofr_F       | ofr_I       | ofr_D       | ofr_E       | ofr_B       | ofr_C       | -           |
| 3504fbcd34474688ac705a18dbf6f9f9 | ofr_F       | ofr_

|                                  | ofr_rec_0   | ofr_rec_1   | ofr_rec_2   | ofr_rec_3   | ofr_rec_4   | ofr_rec_5   | ofr_rec_6   |
|:---------------------------------|:------------|:------------|:------------|:------------|:------------|:------------|:------------|
| 42de30f65a554db2b2928ab318de7390 | ofr_G       | ofr_B       | ofr_F       | ofr_I       | ofr_H       | ofr_C       | ofr_D       |
| d3c01d3f66d142df959c19fb8bef8af9 | ofr_G       | ofr_B       | ofr_I       | ofr_H       | ofr_A       | ofr_D       | -           |
| 02e2211319524b9e8bd09794d00d8513 | ofr_G       | ofr_B       | ofr_I       | ofr_C       | ofr_J       | ofr_E       | -           |
| 1a43f12c497f4ea0aa3897252b84d3dd | ofr_G       | ofr_F       | ofr_H       | ofr_A       | ofr_C       | ofr_D       | ofr_J       |
| 25c89cd833a04d51b76510b1ae335476 | ofr_B       | ofr_F       | ofr_I       | ofr_C       | ofr_D       | ofr_E       | -           |
| 3504fbcd34474688ac705a18dbf6f9f9 | ofr_B       | ofr_F       | ofr_I       | ofr_H       | ofr_J       | ofr_E       | -           |
| 9d9762e2f526402aa214e18be86ecb0f | ofr_G       | ofr_I       | ofr_H       | ofr_D       | ofr_J       | ofr_E       | -           |
| cd0cb9d1599c47a0bb69c49d19c2117f | ofr_G       | ofr_F       | ofr_H       | ofr_A       | ofr_C       | ofr_E       | -           |
| e2a8952a0a504021995b8c609ebd6ded | ofr_G       | ofr_I       | ofr_H       | ofr_A       | ofr_J       | ofr_E       | -           |
| 2e792741fcd548889f45c3399a1d6f69 | ofr_B       | ofr_H       | ofr_A       | ofr_C       | ofr_E       | -           | -           |

In [47]:
print(user_recom.to_markdown()) 

|                                  | ofr_rec_0   | ofr_rec_1   | ofr_rec_2   |
|:---------------------------------|:------------|:------------|:------------|
| 42de30f65a554db2b2928ab318de7390 | ofr_I       | ofr_D       | ofr_B       |
| d3c01d3f66d142df959c19fb8bef8af9 | ofr_G       | ofr_B       | ofr_H       |
| 02e2211319524b9e8bd09794d00d8513 | ofr_J       | ofr_E       | ofr_C       |
| 1a43f12c497f4ea0aa3897252b84d3dd | ofr_D       | ofr_J       | ofr_C       |
| 25c89cd833a04d51b76510b1ae335476 | ofr_F       | ofr_I       | ofr_E       |
| 3504fbcd34474688ac705a18dbf6f9f9 | ofr_E       | ofr_B       | ofr_H       |
| 9d9762e2f526402aa214e18be86ecb0f | ofr_D       | ofr_E       | ofr_H       |
| cd0cb9d1599c47a0bb69c49d19c2117f | ofr_G       | ofr_F       | ofr_E       |
| e2a8952a0a504021995b8c609ebd6ded | ofr_G       | ofr_A       | ofr_H       |
| 2e792741fcd548889f45c3399a1d6f69 | ofr_B       | ofr_H       | ofr_C       |
